##### 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("hw3_genesT.csv", header = None)

In [3]:
X, y = data.iloc[:, 0:1000], data[1000]

## Clustering in search of perfect separation

In [4]:
from sklearn.cluster import KMeans

In [5]:
kmeanclus = KMeans(n_clusters=2).fit(X)

#### Push cluster and actual class into dataframe to see if cluster separated perfectly

In [6]:
cluster_results = pd.DataFrame()
cluster_results["Cluster"], cluster_results["Class"] = kmeanclus.labels_, y 
cluster_results
# indeed, they cluster perfectly

,Cluster,Class
0,1,H
1,1,H
2,1,H
3,1,H
4,1,H
5,1,H
6,1,H
7,1,H
8,1,H
9,1,H


### feature selection to establish feature matrix, X.

In [7]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_newl1 = model.transform(X)
X_newl1.shape

(40, 1)

In [8]:
X, y = data.iloc[:, 0:1000], data[1000]

lsvc = LinearSVC(C=0.01, penalty="l2", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_newl2 = model.transform(X)
X_newl2.shape

(40, 424)

### results of l1 & l2 penalty for feature selection

When applying the L1 penalty for feature selection we are in search of a sparse solution that is able to map X to y. The L1 - regularized loss function is given by  F(x)=f(x)+λ||x||1, a non-smooth function. The L1 penalty will remove one of two variables that are highly correlated, which leads to sparsity. Alternatively the L2 penalty will keep both variables instead constraining the of these variables coefficients, thus reducing the variance in our predictions. 

The variables are removed due to the point at which the tangent line touches the L1 and L2 norm. In a two-dimensional L1 space we see that at this point the x2 or x1 must be zero; in the L2 space there is no such contraint. 

In this case the L1 penalty used to select features drives us to a one-dimensional predictor space. The L2 penalty reduces the dimensions but not nearly as much.

In most cases, the L1 penalty would reduce the dimensions and eliminate the risk of overfitting. While the L2 penalty would reduce risk of overfitting without a loss in prediction power by not eliminating features of our predictor space. That is to say that the L1 penalty is likely to hurt predictive power due to the loss of features, in this case a drastic loss.

In this case the L1 penalty reduces our feature space to one-dimension. This feature space is intriguing because it seems that a one-dimensional space is more likely to be prone to overfitting, the direct problem trying to be solved by the L1 penalty. The L2 penalty just about cut our feature space in half thus reducing variance in our predictions while not shrinking our predictor space nearly as much.

I would like to use the L1 penalty and the L2 penalty in a pipeline with a random forest model and estimate prediction error produced by each method. I will also build a random forest that does not have its features preprocessed, which serves as a control.  

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=57)

## Fit Random Forest

#### Build model using L2 for feature selection

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [11]:
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l2", dual=False))),
  ('classification', RandomForestClassifier())
])
clf.fit(X_train, y_train)

Pipeline(steps=[('feature_selection', SelectFromModel(estimator=LinearSVC(C=0.01, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
        prefit=False, thres...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [12]:
clf.score(X_test, y_test)

0.90000000000000002

#### Building model using L1 for feature selection

In [13]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_newl1 = model.transform(X)
X_newl1.shape

(40, 1)

In [14]:
X_trainl1, X_testl1, y_trainl1, y_testl1 = train_test_split(X_newl1, y, test_size=0.5, random_state=57)

In [15]:
# one-dimensional feature space
X_trainl1.shape

(20, 1)

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf_modl1=RandomForestClassifier()

In [17]:
rf_modl1.fit(X_trainl1, y_trainl1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
rf_modl1.score(X_testl1, y_testl1)

0.84999999999999998

In [19]:
rf_modl1.predict_proba(X_testl1)

array([[ 1. ,  0. ],
       [ 0. ,  1. ],
       [ 1. ,  0. ],
       [ 0. ,  1. ],
       [ 0. ,  1. ],
       [ 1. ,  0. ],
       [ 0. ,  1. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.9,  0.1],
       [ 1. ,  0. ],
       [ 0.2,  0.8],
       [ 0. ,  1. ],
       [ 0. ,  1. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 0.5,  0.5],
       [ 0.2,  0.8],
       [ 1. ,  0. ]])

#### Build model using no feature selection

In [20]:
rf_mod_no_preprocess=RandomForestClassifier()
rf_mod_no_preprocess.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
rf_mod_no_preprocess.score(X_test, y_test)

1.0

### Recap of random forest before moving on to linear modeling with a logistic regression

The random forest preprocessed with L2 and a random forest without preprocessing perform similar in predicting the classes. While the L1 penalty is performs the worst.


# Going linear with a Logistic Regression - w/ L1, L2, and elastic net

The next thing I will do is build a logistic regression using the L1 penalty, L2 penalty, and Elastic Net. Elastic Net uses the optimal balance between the L1 and L2 regularization terms while only adding the computational cost of optimizing another hyperparameter.

#### Building a logisitic regression with L1 penalty

In [22]:
import numpy as np
from sklearn.linear_model import LogisticRegression

In [23]:
logl1= LogisticRegression(penalty='l1')

In [24]:
logl1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [25]:
logl1.score(X_test, y_test)

0.84999999999999998

#### Building a logistic regression with L2 penalty

In [26]:
logl2= LogisticRegression(penalty='l2')

In [27]:
logl2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
logl2.score(X_test, y_test)

0.69999999999999996

#### Building logistic regression with elastic net

In [29]:
from sklearn.linear_model import SGDClassifier

In [30]:
log_en = SGDClassifier(loss='log', penalty='elasticnet', alpha=0.0001, l1_ratio=0.15)

In [31]:
log_en.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [32]:
log_en.score(X_test, y_test)

0.80000000000000004

## Recap thus far

Logistic with L1 penalty out performs both elastic net and L2 penalty applied with logistic regression. But when comparing the results of this linear classifier with our non-linear random forest we get different results. The random forest performs better overall.

In [33]:
# storing the predictions from the best non-linear model - random forest with L2
y_hat_rfl2 = clf.predict(X_test)
y_hat_rfl2

array(['H', 'U', 'U', 'U', 'U', 'H', 'U', 'H', 'H', 'U', 'H', 'H', 'U',
       'U', 'H', 'H', 'H', 'H', 'U', 'H'], dtype=object)

In [34]:
# storing the predictions from the best linear model - logistic regression with L1 penalty
y_hat_lgrl1 = logl1.predict(X_test)
y_hat_lgrl1

array(['U', 'U', 'U', 'U', 'U', 'U', 'U', 'H', 'H', 'U', 'H', 'U', 'U',
       'U', 'H', 'H', 'U', 'U', 'U', 'H'], dtype=object)

In [35]:
from sklearn.metrics import classification_report
target_names = ['Healthy', 'Unhealthy']
print("-------------------------------------------------------------")
print("-Random Forest w/ L2 feature selection classification report-")
print("-------------------------------------------------------------")
print(classification_report(y_test, y_hat_rfl2, target_names=target_names))
print("\n")
print("-------------------------------------------------------------")
print("------Logistic Regression w/ Lasso classification report-----")
print("-------------------------------------------------------------")
print(classification_report(y_test, y_hat_lgrl1, target_names=target_names))

-------------------------------------------------------------
-Random Forest w/ L2 feature selection classification report-
-------------------------------------------------------------
             precision    recall  f1-score   support

    Healthy       0.82      1.00      0.90         9
  Unhealthy       1.00      0.82      0.90        11

avg / total       0.92      0.90      0.90        20



-------------------------------------------------------------
------Logistic Regression w/ Lasso classification report-----
-------------------------------------------------------------
             precision    recall  f1-score   support

    Healthy       1.00      0.67      0.80         9
  Unhealthy       0.79      1.00      0.88        11

avg / total       0.88      0.85      0.84        20



## Mid-point

The random forest classifier with L2 used for features selection gives better predictions than the logistic regression with a lasso (L1) penalty. 

Let's move foward with the random forest and see if other non-linear techniques can boost our ability to classify.

# Building on our results with AdaBoost classifier

The AdaBoost classifier iteratively builds what are know as weak learners. A weak learner is trained on a subset of the features space and is constrained with how deep it may grow, i.e., we limit the number of splits in can make. The first weak learner classifies each instance of our training set recording the instances which it got right and wrong. The instances that are classified as wrong are given weights that allow the next weak learner to "focus" on correctly classifying the instances the weak learner that came before got wrong. The process continues in this interative way where the new weak learner will not be built until after the errors of the weak learner before it are noted and given weights. The weak learners are then used to predict the training set and the majority vote for class 0 or 1 are used to classify the instances in our training set.

In [36]:
from sklearn.ensemble import AdaBoostClassifier

In [37]:
ada = AdaBoostClassifier(base_estimator=None, n_estimators=50, 
                   learning_rate=1.0, algorithm='SAMME.R', 
                   random_state=None)

In [38]:
ada.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [39]:
ada.score(X_test, y_test)

0.80000000000000004

#### Pipeline: L2 feature selection and AdaBoost

In [40]:
adaL2 = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l2", dual=False))),
  ('classification', AdaBoostClassifier())
])
adaL2.fit(X_train, y_train)

Pipeline(steps=[('feature_selection', SelectFromModel(estimator=LinearSVC(C=0.01, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
        prefit=False, threshold=None)), ('classification', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None))])

In [41]:
adaL2.score(X_test, y_test)

0.80000000000000004

In [42]:
y_hat_ada = ada.predict(X_test)
y_hat_adaL2 = adaL2.predict(X_test)

### AdaBoost recap

In [43]:
target_names = ['Healthy', 'Unhealthy']
print("-------------------------------------------------------------")
print("-----------------AdaBoost classification report--------------")
print("-------------------------------------------------------------")
print(classification_report(y_test, y_hat_ada, target_names=target_names))
print("\n")
print("-------------------------------------------------------------")
print("Pipeline:  L2 feature select & AdaBoost classification report")
print("-------------------------------------------------------------")
print(classification_report(y_test, y_hat_adaL2, target_names=target_names))

-------------------------------------------------------------
-----------------AdaBoost classification report--------------
-------------------------------------------------------------
             precision    recall  f1-score   support

    Healthy       0.78      0.78      0.78         9
  Unhealthy       0.82      0.82      0.82        11

avg / total       0.80      0.80      0.80        20



-------------------------------------------------------------
Pipeline:  L2 feature select & AdaBoost classification report
-------------------------------------------------------------
             precision    recall  f1-score   support

    Healthy       0.73      0.89      0.80         9
  Unhealthy       0.89      0.73      0.80        11

avg / total       0.82      0.80      0.80        20



### Let's provide AdaBoost different features

The AdaBoost algorithm can be prone to outliers and noisy data due to its processes at each iteration. If outliers and noise is found in our data the weak learners will adjust to fit such noise at the cost of missing our signal. Once the model is used to predict on test data it will have trouble because it has fit noise and will subsequently fail to map our features to the true binomial distribution of the response variable. As we saw before L2 feature selection leaves us with large feature space (p > 400) which could be contributing to the relative error and instability of the predictions.

In order to reduce our feature space let's use the random forest from earlier to identify variables based on their importance in the model.

#### Construct random forest without pipeline in order to extract feature importances

In [44]:
lsvc = LinearSVC(C=0.01, penalty="l2", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_newL2 = model.transform(X)
X_newL2.shape

(40, 424)

In [45]:
X_trainL2, X_testL2, y_trainL2, y_testL2 = train_test_split(X_newL2, y, test_size=0.5, random_state=57)

In [46]:
from sklearn.ensemble import RandomForestClassifier
rf_modL2=RandomForestClassifier()

In [47]:
rf_modL2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [48]:
imp_feat = rf_modL2.feature_importances_

### Extracting most important features

In [49]:
index_imp_feats = []
for i in range(0,len(imp_feat)):
    if imp_feat[i] != 0.0:
        index_imp_feats.append(i)
print("Columns of most important variables:", index_imp_feats)

Columns of most important variables: [506, 507, 508, 514, 524, 547, 548, 564, 578, 588, 589]


In [50]:
# store subset of X_train and X_test
# only made up of most important features from random forest
X_imp_feats_train = X_train.ix[:, [x for x in index_imp_feats]]
X_imp_feats_test = X_test.ix[:, [x for x in index_imp_feats]]

### Building AdaBoost model with most important features

In [51]:
ada_imp_feats = AdaBoostClassifier(base_estimator=None, n_estimators=300, 
                   learning_rate=1.0, algorithm='SAMME.R', 
                   random_state=80)

In [52]:
ada_imp_feats.fit(X_imp_feats_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=300, random_state=80)

In [53]:
ada_imp_feats.score(X_imp_feats_test, y_test)

0.75

### Building random forest with only most important features

In [54]:
rf_imp_feats = RandomForestClassifier()

In [55]:
rf_imp_feats.fit(X_imp_feats_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [56]:
rf_imp_feats.score(X_imp_feats_test, y_test)

1.0

### Building random forest & L2 pipeline starting with only important features

In [57]:
rf_imp_feats_pipe = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l2", dual=False))),
  ('classification', RandomForestClassifier())
])
rf_imp_feats_pipe.fit(X_imp_feats_train, y_train)

Pipeline(steps=[('feature_selection', SelectFromModel(estimator=LinearSVC(C=0.01, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
        prefit=False, thres...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [58]:
rf_imp_feats_pipe.score(X_imp_feats_test, y_test)

0.90000000000000002

## Recap - to this point

The results are straightforward we get an improvment in the AdaBoost algorithm and the random forest when we use a subset of the most important 15 features. But this is not good enough. Let's take the *best of the best* features and see how our predictions perform with AdaBoost and random forest. Here I will use the top 5 important variables.

In [59]:
X_top_feats_train = X_train.ix[:, [x for x in index_imp_feats[0:5]]]
X_top_feats_test = X_test.ix[:, [x for x in index_imp_feats[0:5]]]

In [60]:
# first three rows of our new training set
# we see top five imp features columns from original data
X_top_feats_train.head(3)

,506,507,508,514,524
18,-1.286128,1.405476,-1.477960,0.520690,0.345878
37,2.659982,3.364177,2.793809,2.149940,1.507902
4,-0.873723,-1.537878,0.387909,2.098149,0.062585


In [61]:
ada_top_feats = AdaBoostClassifier(base_estimator=None, n_estimators=50, 
                   learning_rate=1.0, algorithm='SAMME.R', 
                   random_state=82)

In [62]:
ada_top_feats.fit(X_top_feats_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=82)

In [63]:
ada_top_feats.score(X_top_feats_test, y_test)

0.90000000000000002

In [64]:
rf_top_feats = RandomForestClassifier()

In [65]:
rf_top_feats.fit(X_top_feats_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [66]:
rf_top_feats.score(X_top_feats_test, y_test)

0.94999999999999996

# Part 2

## classifying cancer types

In [67]:
cancdf = pd.read_csv("cancer_data_merged.csv")

In [68]:
cancdf.columns

Index(['Unnamed: 0', 'Patients', 'Class', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6',
       'X7',
       ...
       'X6821', 'X6822', 'X6823', 'X6824', 'X6825', 'X6826', 'X6827', 'X6828',
       'X6829', 'X6830'],
      dtype='object', length=6833)

In [69]:
X_canc, y_canc = cancdf.ix[:, 3:], cancdf[[2]]

In [70]:
X_trainc, X_testc, y_trainc, y_testc = train_test_split(X_canc, y_canc, test_size=0.5, random_state=90)

In [71]:
log_2 = SGDClassifier(loss='log', penalty='elasticnet', alpha=0.0001, l1_ratio=0.15)
log_2.fit(X_trainc, y_trainc)
f1score = log_2.score(X_testc, y_testc)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [72]:
print("----Using logistic regression with elastic net regularization----")
print("f1-score: " + str(f1score))
print("-----------------------------------------------------------")
print("\n")
print("Cancer type, " + "most important feature (by column number)")
print("-----------------------------------------------------------")
print("-----------------------------------------------------------")
for i in range(0, log_2.coef_.shape[0]):
    top15_indices = np.argsort(log_2.coef_[i])[-15:]
    if str(y_trainc['Class'].unique()[i]) == "NSCLC":
        print(str(y_trainc['Class'].unique()[i]).upper() + ": " + str(top15_indices))
    elif str(y_trainc['Class'].unique()[i]) == "CNS":
        print(str(y_trainc['Class'].unique()[i]).upper() + ": " + str(top15_indices))
    else:
        print(str(y_trainc['Class'].unique()[i]).title() + ": " + str(top15_indices))

----Using logistic regression with elastic net regularization----
f1-score: 0.666666666667
-----------------------------------------------------------


Cancer type, most important feature (by column number)
-----------------------------------------------------------
-----------------------------------------------------------
Breast: [3863 3827 6687  162 5889 4447 3826 2748 5877 6603 5890 5891  515 6716  280]
Renal: [4041 5891 3690  805 6409 5868 6645 3559 6688 3542 3862 3863 3247 6687 3437]
CNS: [3923 5892  232 3559 5384 5382 3862 4246 2008  285 5456 3524 5383 5455 4247]
Colon: [6687 3828 3826 3225 1048 3690 6688 3864 6457 3827 3862 3863 3247 3437 4100]
Ovarian: [5528 6688 3825 5868 4573 4100 3863 5983 3862 3826 3827 3690  805 6687 3867]
Melanoma: [6277 5643 6355  514 5868 3559 5837 5893 4279 6687  243 6462 5533 4105 3542]
NSCLC: [ 805 3501 6688  515 5650  516 5585  514 5790 5936 5983 6017 3437 6687  132]
Leukemia: [5528 3691  200 3827 3826 6017 5527 5867 4573 3690 6645 3542 3437 5868

In [73]:
important_feats = {}
for i in range(0, log_2.coef_.shape[0]):
    top15_indices = np.argsort(log_2.coef_[i])[-15:]
    for feat in top15_indices:
        if feat not in important_feats:
            important_feats[feat] = 0
        important_feats[feat] += 1

In [74]:
dict_df = pd.DataFrame([important_feats])
dict_df = dict_df.T
dict_df["Column"] = dict_df.index
dict_df["Count"] = dict_df[0]
del dict_df[0]
dict_df.head(6)

,Column,Count
132,132,1
162,162,1
195,195,1
200,200,1
232,232,1
243,243,1


### Top 10 columns by number of cancers, for which, they are a top 15 feature as determined by the coefficient in logistic regression.

For example, the features in columns 6687 and 3437 are in the top 15 features for 5 of the 9 cancers.

In [75]:
dict_df.sort_values("Count", ascending=False).values[0:10]

array([[6687,    6],
       [3690,    4],
       [3437,    4],
       [3862,    4],
       [5868,    4],
       [3827,    4],
       [3826,    4],
       [3863,    4],
       [6688,    4],
       [ 805,    3]])